In [ ]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from openai import OpenAI
from pprint import pprint

# 環境変数の取得
load_dotenv("../.env")

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

In [ ]:
# モデル名
MODEL_NAME = "gpt-4o-mini"

# メッセージの設定
message = "言語モデルを使う上でのポイントは"

# APIへリクエスト
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "user", "content": message},
    ]
)

In [ ]:
"""
OpenAI ChatCompletion レスポンス可視化（Notebook向け）

概要:
    OpenAI API のレスポンスを Notebook 上で読みやすく表示するユーティリティです。
    response.model_dump() 可能なオブジェクト、または dict を入力として受け取り、
    以下の順で表示します。

入力（Input）:
    response_obj: Any | dict
        - OpenAI のレスポンスオブジェクト（model_dump() を持つもの）または dict。
        - 例:
            response_obj = response
            response_obj = response.model_dump()

出力（Output）:
    戻り値: None
    Notebook への表示（IPython.display.display）:
        1) Raw JSON（整形表示）
        2) 出力補足テーブル（フィールド / 値 / 説明）
           - JSONの出現順（dictの順序）を最優先して表示
           - list は「配列そのもの（prefix）」→「先頭要素の中身（prefix[]...）」の順で表示
           - choices[].message.content は表ではプレースホルダ表示し、本文は別枠で表示
           - 説明は FIELD_DESCRIPTIONS を優先し、未登録は auto_description() で自動生成
        3) choices[].message.content（回答本文）を Markdown 表示（箇条書き等を維持）

仕様メモ:
    - 「JSON順」を崩さないため、並べ替えは行いません（discover_paths_in_order の順を採用）。
    - もし JSON に無いが強制的に表示したいパスがある場合のみ FIELD_ORDER_FALLBACK に追加できます。
"""

import json
from typing import Any, Dict, List, Optional, Tuple, Set, Union
from IPython.display import Markdown, HTML, display


# 「もしJSONに無いが、表示したい」ものがあればここに入れる（基本は少なめ推奨）
# ※ JSON順を崩さないため、このリストは “不足分だけ末尾に補う” ために使う
FIELD_ORDER_FALLBACK: List[str] = [
    # 例: "usage.total_tokens",
]

# 既知フィールドの説明（無ければ自動生成）
FIELD_DESCRIPTIONS: Dict[str, str] = {
    # --- Top-level ---
    "id": "このレスポンスの一意な識別子（追跡・問い合わせ用）",
    "created": "レスポンス生成時刻（UnixTime: 秒）",
    "model": "実際に応答生成に使われたモデル名（バージョン含むことあり）",
    "object": "レスポンス種別（このAPIが返すオブジェクトタイプ）",
    "service_tier": "処理レーン/サービス階層（default 等）",
    "system_fingerprint": "モデル実装の指紋（同一モデルでも内部更新を識別する用途）",

    # --- choices ---
    "choices": "生成結果の候補配列（通常 choices[0] を採用）",
    "choices[].finish_reason": "生成が止まった理由（stop/length/tool_calls 等）",
    "choices[].index": "choices 配列内の候補番号（0始まり）",
    "choices[].logprobs": "トークン確率情報（有効化したときだけ付く）",

    # --- message ---
    "choices[].message": "生成メッセージ本体（role/content 等）",
    "choices[].message.role": "発話者ロール（assistant 等）",
    "choices[].message.content": "生成テキスト本文（表では別枠表示）",
    "choices[].message.refusal": "拒否の理由/種別（拒否時のみ。通常 null）",
    "choices[].message.annotations": "注釈情報（機能/モデルにより付与）",
    "choices[].message.audio": "音声出力メタ情報（音声出力時のみ）",
    "choices[].message.function_call": "旧式の関数呼び出し情報（互換用。通常 null）",
    "choices[].message.tool_calls": "ツール呼び出し情報（ツール利用時のみ）",

    # --- usage ---
    "usage": "トークン使用量の集計（課金/最適化の基本）",
    "usage.prompt_tokens": "入力側トークン数（プロンプトの長さ）",
    "usage.completion_tokens": "出力側トークン数（回答の長さ）",
    "usage.total_tokens": "合計トークン数（prompt + completion）",

    # --- usage details (prompt) ---
    "usage.prompt_tokens_details": "入力トークンの内訳（対応時のみ）",
    "usage.prompt_tokens_details.cached_tokens": "キャッシュヒットした入力トークン数（該当時）",
    "usage.prompt_tokens_details.audio_tokens": "入力の音声トークン数（音声入力時）",

    # --- usage details (completion) ---
    "usage.completion_tokens_details": "出力トークンの内訳（対応時のみ）",
    "usage.completion_tokens_details.reasoning_tokens": "推論に割り当てられたトークン数（対応時）",
    "usage.completion_tokens_details.accepted_prediction_tokens": "採用された予測トークン数（対応時）",
    "usage.completion_tokens_details.rejected_prediction_tokens": "棄却された予測トークン数（対応時）",
    "usage.completion_tokens_details.audio_tokens": "出力の音声トークン数（音声出力時）",
}



def _to_dict(response_obj: Union[Any, Dict[str, Any]]) -> Dict[str, Any]:
    """
    response_obj を dict に正規化する。

    Args:
        response_obj:
            - dict または model_dump() を持つオブジェクト

    Returns:
        dict 化したレスポンス

    Raises:
        TypeError: model_dump() を持たず dict でもない場合
    """
    if isinstance(response_obj, dict):
        return response_obj
    model_dump = getattr(response_obj, "model_dump", None)
    if callable(model_dump):
        return model_dump()
    raise TypeError("response_obj は model_dump() を持つオブジェクトか dict である必要があります。")


def display_raw_json(data: Dict[str, Any]) -> None:
    """
    レスポンス全体を整形JSONとして表示する（Markdownコードブロック）。

    Args:
        data: レスポンスdict
    """
    pretty = json.dumps(data, indent=2, ensure_ascii=False)
    display(Markdown("## 🧾 Raw JSON"))
    display(Markdown(f"```json\n{pretty}\n```"))


def escape_html(s: Any) -> str:
    """
    HTML表示用の最低限のエスケープ。

    Args:
        s: 任意の値

    Returns:
        HTMLエスケープ済み文字列
    """
    if s is None:
        return ""
    s = str(s)
    return s.replace("&", "&amp;").replace("<", "&lt;").replace(">", "&gt;")


def format_value_for_table(v: Any, max_len: Optional[int] = 180) -> str:
    """
    テーブルの「値」表示用に整形する（HTML文字列）。

    仕様:
        - dict/list は JSON 文字列化（ensure_ascii=False）
        - 改行 → <br>
        - HTMLエスケープ（ただし <br> は復活）
        - 長文は max_len で省略（None なら省略しない）

    Args:
        v: 表示する値
        max_len: 最大表示文字数（Noneなら省略しない）

    Returns:
        HTMLとして安全な表示用文字列
    """
    if v is None:
        s = ""
    elif isinstance(v, (dict, list)):
        s = json.dumps(v, ensure_ascii=False)
    else:
        s = str(v)

    s = s.replace("\n", "<br>")
    s = escape_html(s).replace("&lt;br&gt;", "<br>")  # <br>だけ復活

    if max_len is not None and len(s) > max_len:
        s = s[:max_len] + "…"
    return s


def get_value_by_path(data: Any, path: str) -> Any:
    """
    path で指定された値を取得する。

    ルール:
        - "choices[]" のような [] は配列の先頭要素を代表として扱う
          例: choices[].message.role は choices[0].message.role を参照する

    Args:
        data: ルートオブジェクト（dict想定だが、途中でlist/scalarもありうる）
        path: ドット区切りパス（例: "usage.total_tokens", "choices[].message.content"）

    Returns:
        該当値（存在しない/辿れない場合は None）
    """
    cur = data
    for part in path.split("."):
        is_list = part.endswith("[]")
        key = part[:-2] if is_list else part

        if isinstance(cur, list):
            cur = cur[0] if cur else None

        if cur is None:
            return None
        if not isinstance(cur, dict):
            return None

        cur = cur.get(key)

        if is_list and isinstance(cur, list):
            cur = cur[0] if cur else None
    return cur


def discover_paths_in_order(data: Any, prefix: str = "") -> List[str]:
    """
    JSONの出現順（dictの順序）をできるだけ維持してフィールドパスを収集する。

    ルール:
        - dict: キー順に再帰（prefix.key）
        - list:
            1) prefix（配列そのもの）を先に paths に追加
            2) 先頭要素があれば prefix[] を付けて先頭要素を再帰（中身のフィールドを拾う）
        - scalar: prefix を追加

    Args:
        data: 任意のJSONライクなデータ（dict/list/scalar）
        prefix: 現在のパス

    Returns:
        フィールドパスのリスト（JSON順に近い）
    """
    paths: List[str] = []

    if isinstance(data, dict):
        for k, v in data.items():
            p = f"{prefix}.{k}" if prefix else k
            paths.extend(discover_paths_in_order(v, p))
        return paths

    if isinstance(data, list):
        if prefix:
            paths.append(prefix)  # 配列そのもの
            if data:
                paths.extend(discover_paths_in_order(data[0], prefix + "[]"))
        return paths

    if prefix:
        paths.append(prefix)
    return paths


def auto_description(path: str, value: Any) -> str:
    """
    FIELD_DESCRIPTIONS に無いフィールドの説明を自動生成する。

    方針:
        - 末尾キー名（role/content/tokens 等）でなるべく意味のある説明を付ける
        - それでも難しい場合は型（None/int/str/list/dict）から推測

    Args:
        path: フィールドパス
        value: 実値（Noneの可能性あり）

    Returns:
        自動生成した説明文
    """
    last = path.split(".")[-1]
    if last.endswith("[]"):
        last = last[:-2]

    # キー名ベース（よくあるものを優先）
    if last == "content":
        return "本文テキスト"
    if last == "role":
        return "発話者ロール"
    if last == "index":
        return "配列のインデックス"
    if last == "refusal":
        return "拒否に関する情報（通常は null）"
    if last == "annotations":
        return "注釈情報（通常は空/なし）"
    if last == "audio":
        return "音声関連情報（音声入出力時のみ）"
    if last in ("function_call", "tool_calls"):
        return "関数/ツール呼び出し情報（該当時のみ）"
    if last.endswith("_tokens"):
        return "トークン数"
    if last.endswith("_token"):
        return "トークン"

    # 型ベース
    if value is None:
        return "（値なし / null）"
    if isinstance(value, bool):
        return "真偽値"
    if isinstance(value, int):
        return "数値"
    if isinstance(value, float):
        return "数値（小数）"
    if isinstance(value, str):
        return "文字列"
    if isinstance(value, list):
        return "配列"
    if isinstance(value, dict):
        return "オブジェクト"

    return "（自動説明）"


def build_rows_json_order(data: Dict[str, Any]) -> Tuple[List[Tuple[str, str, str]], str]:
    """
    JSON順を最優先にテーブル行を作る。

    仕様:
        - discover_paths_in_order の順番をそのまま採用（並べ替えなし）
        - FIELD_ORDER_FALLBACK は「JSONに無いが表示したい」場合のみ末尾に補う
        - choices[].message.content は表ではプレースホルダ、本文として別返却

    Args:
        data: レスポンスdict

    Returns:
        rows: (field_path, value_html, description) のリスト
        content_text: choices[].message.content の生本文（無ければ ""）
    """
    discovered = discover_paths_in_order(data)
    discovered_set: Set[str] = set(discovered)

    ordered = list(discovered)
    for p in FIELD_ORDER_FALLBACK:
        if p not in discovered_set:
            ordered.append(p)

    rows: List[Tuple[str, str, str]] = []
    content_text = ""

    for p in ordered:
        val = get_value_by_path(data, p)

        desc = FIELD_DESCRIPTIONS.get(p)
        if desc is None:
            desc = auto_description(p, val)

        if p == "choices[].message.content":
            if isinstance(val, str):
                content_text = val
            value_html = "（下に本文を表示）"
        else:
            value_html = format_value_for_table(val)

        rows.append((p, value_html, desc))

    return rows, content_text


def render_html_table(rows: List[Tuple[str, str, str]]) -> None:
    """
    補足テーブルを HTML で表示する。

    表示仕様:
        - フィールド列は左寄せ + monospace
        - ゼブラ表示
        - 横スクロール対応

    Args:
        rows: (field, value_html, desc) の行リスト
    """
    html: List[str] = []
    html.append('<h2>📘 出力補足（JSON順・動的）</h2>')
    html.append('<div style="overflow-x:auto;">')
    html.append('<table style="border-collapse:collapse; width:100%;">')
    html.append("<thead><tr>")
    html.append('<th style="text-align:left; padding:8px 10px; border-bottom:1px solid #ddd;">フィールド</th>')
    html.append('<th style="text-align:left; padding:8px 10px; border-bottom:1px solid #ddd;">値</th>')
    html.append('<th style="text-align:left; padding:8px 10px; border-bottom:1px solid #ddd;">説明</th>')
    html.append("</tr></thead><tbody>")

    zebra = False
    for field, value_html, desc in rows:
        bg = "#fafafa" if zebra else "white"
        zebra = not zebra
        html.append(f'<tr style="background:{bg};">')
        html.append(
            f'<td style="text-align:left; vertical-align:top; padding:8px 10px; font-family:monospace;">'
            f'{escape_html(field)}</td>'
        )
        html.append(f'<td style="vertical-align:top; padding:8px 10px;">{value_html}</td>')
        html.append(f'<td style="vertical-align:top; padding:8px 10px;">{escape_html(desc)}</td>')
        html.append("</tr>")

    html.append("</tbody></table></div>")
    display(HTML("\n".join(html)))


def display_content_block(content: str) -> None:
    """
    choices[].message.content（回答本文）を Markdown 表示する。

    Args:
        content: 本文文字列
    """
    display(Markdown("## ✨ choices[].message.content（回答本文）"))
    display(Markdown(content or ""))


def display_response_analysis_json_order(response_obj: Union[Any, Dict[str, Any]]) -> None:
    """
    エントリーポイント。

    入力:
        response_obj: Any | dict
            - model_dump() を持つオブジェクト、または dict

    出力:
        戻り値 None
        Notebook 表示:
            1) Raw JSON
            2) JSON順の動的補足テーブル
            3) 本文（content）

    Args:
        response_obj: レスポンスオブジェクト
    """
    data = _to_dict(response_obj)

    display_raw_json(data)

    rows, content = build_rows_json_order(data)
    render_html_table(rows)

    display_content_block(content)


# ================================
# 実行
# ================================
display_response_analysis_json_order(response)
